#Creator: Chike Uduku
#Created: 03/22/2019
#Revision: 1.0

### Heroes Of Pymoli Data Analysis
* The game "Oathbreaker, Last Hope of the Breaking Storm" appears to have very promising potential. Not only was it the most purchased game (12 purchases), it also generated the most revenue despite being cheaper than some of the other games.

* The highest percentage of purchases (approx. 46.8%) was done by people in the age group of 20 - 24 years. This may be simply down to the category of games used in this dataset, or it may more broadly reflect the perceived notion that age range is unique in the sense that it is one of the only times on the age spectrum where there is a direct correlation between available leisurely time and a person's bank account.

* 84% of players were male and this gender group was responsible for approximately 83.5% of total purchases
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [60]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import warnings

#hide pink warnings
warnings.filterwarnings('ignore')

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
myDF = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [48]:
playerCount = myDF["SN"].nunique() #Get unique list of players
playerCount

576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [52]:
ui = myDF["Item ID"].nunique() #Get unique number of itmems

#Avergae purchase price should be calculated using unique set of Items. Let's drop duplicates by iterating over the Item Id
#column and adding each item ID and price to a dictionary as key-Value pairs. In the future,a better way to do this might
#be to use the drop_duplicates() function of a panda data frame
myDict = {}
for i  in range(0,myDF["Item ID"].count()):
    myDict[myDF.iloc[i,4]] = myDF.iloc[i,6]
#Now that we have our unique set of items, let's make it a dataframe so we can take advantage of some data frame object
#functions
J = pd.DataFrame(myDict, index = range(0,1))
#Let's get the average purchase price
avgPurPrice = J.mean(axis = 1)
#Let's get the number of purchases
numPurchase = myDF["SN"].count()
#Let's get our total revenue value
totalRev = myDF["Price"].sum()
#Average sell price is calculated in a similar manner to average purchase price, but includes duplicates
avgSellPrice = totalRev / numPurchase

#Let's make a data frame containing our results and do some formatting before displaying this result data frame 
rslt_df = pd.DataFrame({"Number of unique items":ui,"Average Purchase Price":avgPurPrice,"Average Sell Price":avgSellPrice, \
              "Number of Purchases":numPurchase,"Total Revenue":totalRev})
rslt_df["Average Purchase Price"] = rslt_df["Average Purchase Price"].map("${:.2f}".format)
rslt_df["Average Sell Price"] = rslt_df["Average Sell Price"].map("${:.2f}".format)
rslt_df["Total Revenue"] = rslt_df["Total Revenue"].map("${:,.2f}".format)
rslt_df

,Number of unique items,Average Purchase Price,Average Sell Price,Number of Purchases,Total Revenue
0,183,$3.04,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [53]:
PlyrsDF = myDF[["SN","Gender"]] #Grab a sub data frame of player name and gender
#some players may appear twice or more times if they made more than one purchase, so drop duplicates to obtain a unique
#data frame of players and their gender
uniquePlyrDF = PlyrsDF.drop_duplicates()

#Now that we have that, let's group our unique data frame by gender
genderGroup = uniquePlyrDF.groupby("Gender")

#Grab the dataframe returned by the count method of our grouped data object.This should give us total counts. Solve for 
#percentage of players and add that series to the returned dataframe to obtain our final result
finalDF= genderGroup.count() #total count
finalDF.columns = ["Total Count"]#rename column to say total  count
finalDF["Percentage of players"] = (finalDF["Total Count"] / finalDF["Total Count"].sum()) * 100 #solve for % of players
finalDF.sort_values(by = "Total Count",ascending = False, inplace = True) #sort in descending order
finalDF["Percentage of players"]= finalDF["Percentage of players"].map("${:.2f}".format) #format % of players column
finalDF

,Total Count,Percentage of players
Gender,,
Male,484,$84.03
Female,81,$14.06
Other / Non-Disclosed,11,$1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [54]:
gndrPur_df = myDF [["SN","Gender","Item ID","Price"]] #grab a sub data frame of player, gender, item iD and price
groupGndrPur = myDF.groupby("Gender") #group our sub data frame by gender

#Create series objects for total count, avrage putchase price,total purchase value and average total purchase per person 
#respectively. Put them in a dictionary with the appropriate label and make a data frame out of the dictionary to display
#our final results
countSer = groupGndrPur["SN"].count() #Purchase count series
totalSer= groupGndrPur["Price"].sum() #Total purchase value series
avgPurSer = totalSer / countSer       #Average purchase price series
avgPurPrsnSer = totalSer / finalDF["Total Count"]#Avergae purchase price per person series
aDict = {"Purchase Count":countSer,"Avergae Purchase Price":avgPurSer,"Total Purchase Value":totalSer, \
         "Avg Total Purchase per Person":avgPurPrsnSer} #add to dictionary
pag_df = pd.DataFrame(aDict)#create data frame from dictionary

#Let's do some formatting before we display our final result
pag_df["Avergae Purchase Price"] = pag_df["Avergae Purchase Price"].map("${:.2f}".format)
pag_df["Total Purchase Value"] = pag_df["Total Purchase Value"].map("${:,.2f}".format)
pag_df["Avg Total Purchase per Person"] = pag_df["Avg Total Purchase per Person"].map("${:.2f}".format)
pag_df

,Purchase Count,Avergae Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [61]:
ageBins = [0,9,14,19,24,29,34,39,120] #bins for age groups
binLabels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"] #labels for age groups
age_df = myDF[["SN","Age"]] #Create sub data frame of of players and age
uniqueAge_df = age_df.drop_duplicates()#Get rid of duplicates to get a unique data frame of players and age

#add column to our unique data frame reflecting what bin the age entry for that row has been placed in and name it 
#"Age Group". 
uniqueAge_df["Age Group"] = pd.cut(uniqueAge_df["Age"],ageBins,labels = binLabels)
#Group data according to age group
ageGrouping = uniqueAge_df.groupby("Age Group")
#Get the data frame returned from the count method of our grouped data object to get total counts for each age group
reportDF = ageGrouping.count()

reportDF = reportDF.rename(columns = {"SN":"Total Count"})#rename column
del(reportDF["Age"])#delete age column as it is not needed
reportDF["Percentage of Players"]= reportDF["Total Count"]/playerCount * 100 #solve for % of players and add to data frame

#Let's do some formatting
reportDF["Percentage of Players"] = reportDF["Percentage of Players"].map("{:.2f}".format)
reportDF

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [62]:
binPD_df = myDF[["Age","Price"]] #get sub data frame of age and price

#find appropriate bin for age entry of each row and add this column aptly named "Age Group" to our sub data frame
binPD_df["Age Group"] = pd.cut(binPD_df["Age"],ageBins,labels = binLabels)
#group data frame by age group
binPD_group = binPD_df.groupby("Age Group")

#From grouped data object, obtain series objects for purchase count, total purchase value and average purchase price
purCntSer = binPD_group["Price"].count()#purchase count
ttlPurValSer =  binPD_group["Price"].sum()#total purchase value
avgPriceSer = ttlPurValSer /purCntSer# average purchase price

#We need to be able to divide total purchase by number of people in each age group to find total purchase per person
#To find number of people in each group, we need a unique set of players in each age group.
uniqueBinDF = myDF[["SN","Age"]].drop_duplicates()# get UNIQUE data frame of players and their ages
uniqueBinDF["Age Group"] = pd.cut(uniqueBinDF["Age"],ageBins,labels = binLabels) #put in bins and add to sub data frame
uniqueBin_group = uniqueBinDF.groupby("Age Group") # group by age group
avgTotPerPsnSer = ttlPurValSer / uniqueBin_group["Age"].count()#solve for average total purchase per person

#create dictionary key value pairs with results
apaDict = {"Purchase Count":purCntSer,"Average Purchase Price":avgPriceSer,"Total Purchase Value":ttlPurValSer, \
          "Avg Total Purchase per Person":avgTotPerPsnSer}
#Obtain data frame from dictionary
rsltDF = pd.DataFrame(apaDict) 
#Let's do some formatting and display result
rsltDF["Average Purchase Price"]= rsltDF["Average Purchase Price"].map("${:.2f}".format)
rsltDF["Avg Total Purchase per Person"]= rsltDF["Avg Total Purchase per Person"].map("${:.2f}".format)
rsltDF["Total Purchase Value"]= rsltDF["Total Purchase Value"].map("${:,.2f}".format)
rsltDF

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [57]:
plyrgrouped_df = myDF.groupby("SN") #group data frame by player names
pCountSer = plyrgrouped_df["SN"].count() #series for purchase count
totPurSer = plyrgrouped_df["Price"].sum()# series for total purchase price
avgSer = totPurSer / pCountSer#solve for Average purchase price series

#Create dictionary of results
dictRslt = {"Purchase Count":pCountSer,"Average Purchase Price":avgSer,"Total Purchase Value":totPurSer}
#Create data frame of results from dictionary
rslt_df = pd.DataFrame(dictRslt)

#sort data frame and do some formatting before displaying results
rslt_df.sort_values(by = 'Total Purchase Value',ascending = False, inplace = True)
rslt_df["Average Purchase Price"] = rslt_df["Average Purchase Price"].map("${:.2f}".format)
rslt_df["Total Purchase Value"] = rslt_df["Total Purchase Value"].map("${:.2f}".format)
rslt_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [58]:
item_df = myDF[["Item ID","Item Name","Price"]] #obtain sub data frame of Item Id, Item name and Price
itemsGroup = item_df.groupby(["Item ID","Item Name"])#group sub data frame by Item ID and Item Name
counts = itemsGroup["Item Name"].count()#Series for Purchase count
totals = itemsGroup["Price"].sum()# series for total purchase value
items = totals / counts # series for Item price

#Create dictionary of results
rsltDict = {"Purchase Count":counts,"Item Price":items,"Total Purchase Value":totals}
#Create a data frame of results from the dictionary
rsltFrame = pd.DataFrame(rsltDict)
#sort result by purchase count in descending order and do some formatting before displaying
rsltFrameCountSort = rsltFrame.sort_values(by = "Purchase Count", ascending = False)
rsltFrameCountSort["Item Price"] = rsltFrameCountSort["Item Price"].map("${:.2f}".format)
rsltFrameCountSort["Total Purchase Value"] = rsltFrameCountSort["Total Purchase Value"].map("${:.2f}".format)
rsltFrameCountSort.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [59]:
#sort result above by Total purchase value in descending order and do some formatting before displaying
rsltFramePurSort = rsltFrame.sort_values(by = "Total Purchase Value", ascending = False)
rsltFramePurSort["Item Price"] = rsltFramePurSort["Item Price"].map("${:.2f}".format)
rsltFramePurSort["Total Purchase Value"] = rsltFramePurSort["Total Purchase Value"].map("${:.2f}".format)
rsltFramePurSort.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
